## Regression Against Time
### The Formation Process of Winners and Losers in Momentum Investing
(https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2610571)

> **p. 3**: Intermediate-term (3-12 months) momentum has been documented by Jegadeesh and Titman (1993, 2001, hereafter JT), while short-term (weekly) and long-term (3-5years) reversals have been documented by Lehmann (1990) and by DeBondt and Thaler (19850, respectively. Various models and theories have benn proposed to explain the coexistence of intermediate-term momentum and long-term reversal. However, most studies have focused primarily on which stocks become winners or losers. This paper develops a model to analyze whether the movement of historical prices is related to future expected returns.

> **p. 4**: This paper captures the idea that past reutrns and the formation process of past returns have a joint effect on future expected returns. We argue that how one stock becomes a winner or loser, that-is, the movement of historical prices-plays an important role in momentum investing. Using a polynomial quadratic model to approximate the nonlinear patter of historical prices, the model shows that as long as two stocks share the same return over th past n-month, the future expected return of tow stocks whos historical prices are convex shaped is not lower than one whose historical prices are concave shaped. In other words, when there are two winner (or loser) stocks, the one with the convex-shaped historical prices will possess higher future expected returns than the one with concave-shaped historical prices.

> **p. 4**: To test teh model empirically, we regress previous daily prices in the ranking period on an oridinal time variable and the square of the ordinal time variable for each stock. The coefficient of the square of the oridianl time variable is denoted as $\gamma$.

In [5]:
# Imports
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import morningstar as mstar
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.factors.morningstar import MarketCap
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.factors import Returns
from zipline.pipeline.factors import DailyReturns

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import time

from quantopian.pipeline.experimental import QTradableStocksUS

In [6]:
def make_pipeline():
    average_day_dv_200 = AverageDollarVolume(window_length = 200)
    market_cap = Fundamentals.market_cap.latest
    price_open = USEquityPricing.open.latest
    price_close = USEquityPricing.close.latest
    volume = USEquityPricing.volume.latest
    sector = Sector()
    daily_returns = DailyReturns(inputs = [USEquityPricing.close])
    
    # Ranked Returns
    returns = Returns(window_length=252)
    ranked_retuns = returns.rank()
    
    # New Factors
    # create a factor of 1 year returns, demeaned by sector, rank, zscore
    factor = (
    Returns(window_length=252).\
    demean(groupby = Sector()).\
    rank().\
    zscore()
    )
    
    # use the newly created factor as an input into SimpleMovingAverage, with window length = 5
    factor_smoothed = (SimpleMovingAverage(inputs = [factor], window_length=5).\
                      rank().\
                      zscore()
                      )
    
    return Pipeline(
        columns = {
            'AverageDollarVolume': average_day_dv_200,
            'MarketCap': market_cap,
            'close_price':price_close,
            'open_price':price_open,
            'volume':volume,
            'sector':sector,
            '1yrReturns': factor,
            '5dAvgReturns': factor_smoothed,
            'ranked_returns':ranked_retuns,
            '1dReturns':daily_returns
        },
        screen = QTradableStocksUS()
    )

In [7]:
# pipeline is run over this time range  and outputs a dataframe indexed by asset name:
start_date = '2017'
end_date = '2019-08-05'

QTU_pipeline2 = run_pipeline(make_pipeline(), start_date, end_date, chunksize=252)

In [8]:
# lets see what stocks had the most gains ytd through August 2nd
leading_ytd_stocks = QTU_pipeline2.sort_values(by=['ranked_returns'], ascending=False)
leading_ytd_stocks.filter(like='2019-08-02', axis = 0).head(25)

1dReturns  1yrReturns  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])   0.018846    1.729817   
                          Equity(42749 [ENPH])   0.069627    1.725988   
                          Equity(32726 [EHTH])  -0.012531    1.720882   
                          Equity(32215 [APPS])   0.027599    1.718330   
                          Equity(50735 [AYX])    0.120985    1.711948   
                          Equity(50477 [IIPR])  -0.036067    1.712586   
                          Equity(48628 [NVTA])   0.011165    1.714501   
                          Equity(50288 [TTD])    0.004899    1.708757   
                          Equity(50411 [RARX])  -0.032021    1.711310   
                          Equity(31341 [ZIOP])  -0.025937    1.705566   
                          Equity(51314 [MDB])    0.004120    1.700460   
                          Equity(50449 [INSG])  -0.048170    1.697908   
                          Equity(50758 [OKTA])   0.017274    1.695993   
                          Equity(4549 [LSCC])   -0.001552    1.693440   
                          Equity(49060 [SHOP])   0.074229    1.692164   
                          Equity(45758 [VCYT])  -0.046610    1.701099   
                          Equity(49460 [NVCR])   0.032223    1.699184   
                          Equity(51961 [SMAR])   0.001202    1.685782   
                          Equity(51832 [ZS])     0.011867    1.685144   
                          Equity(44991 [NSTG])  -0.126945    1.694078   
                          Equity(47321 [CDNA])   0.036314    1.692802   
                          Equity(50077 [TWLO])  -0.008914    1.680676   
                          Equity(48254 [WK])     0.013572    1.671104   
                          Equity(47779 [CYBR])  -0.003242    1.670465   
                          Equity(45429 [FATE])  -0.030413    1.688973   

                                                5dAvgReturns  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])      1.729248   
                          Equity(42749 [ENPH])      1.720529   
                          Equity(32726 [EHTH])      1.723643   
                          Equity(32215 [APPS])      1.719906   
                          Equity(50735 [AYX])       1.707450   
                          Equity(50477 [IIPR])      1.716169   
                          Equity(48628 [NVTA])      1.713055   
                          Equity(50288 [TTD])       1.711187   
                          Equity(50411 [RARX])      1.713678   
                          Equity(31341 [ZIOP])      1.703090   
                          Equity(51314 [MDB])       1.696862   
                          Equity(50449 [INSG])      1.704958   
                          Equity(50758 [OKTA])      1.694993   
                          Equity(4549 [LSCC])       1.678800   
                          Equity(49060 [SHOP])      1.662607   
                          Equity(45758 [VCYT])      1.706204   
                          Equity(49460 [NVCR])      1.695616   
                          Equity(51961 [SMAR])      1.685028   
                          Equity(51832 [ZS])        1.681291   
                          Equity(44991 [NSTG])      1.706827   
                          Equity(47321 [CDNA])      1.702467   
                          Equity(50077 [TWLO])      1.683160   
                          Equity(48254 [WK])        1.676932   
                          Equity(47779 [CYBR])      1.673818   
                          Equity(45429 [FATE])      1.699353   

                                                AverageDollarVolume  \
2019-08-02 00:00:00+00:00 Equity(49607 [AXSM])         1.541528e+07   
                          Equity(42749 [ENPH])         2.810533e+07   
                          Equity(32726 [EHTH])         2.175493e+07   
                          Equity(32215 [APPS])         3.525766e+06   
                          Equity(50735 [AYX])          6.546713e+07   
                          Equity(50477 [IIPR])         2.4

In [9]:
# lets see what stocks had the most gains today
leading_daily_stocks = QTU_pipeline2.sort_values(by=['1dReturns'], ascending = False)
leading_daily_stocks.filter(like='2019-08-02', axis=0).head()

1dReturns  1yrReturns  \
2019-08-02 00:00:00+00:00 Equity(13698 [MYGN])   0.550978    0.827415   
                          Equity(28326 [VNDA])   0.269076   -0.520445   
                          Equity(20359 [EGOV])   0.246968    1.373706   
                          Equity(27817 [SPWR])   0.241574    1.631536   
                          Equity(7130 [STAA])    0.188673    1.092902   

                                                5dAvgReturns  \
2019-08-02 00:00:00+00:00 Equity(13698 [MYGN])     -0.572679   
                          Equity(28326 [VNDA])     -0.971904   
                          Equity(20359 [EGOV])      0.692260   
                          Equity(27817 [SPWR])      1.531816   
                          Equity(7130 [STAA])       0.384589   

                                                AverageDollarVolume  \
2019-08-02 00:00:00+00:00 Equity(13698 [MYGN])         2.276041e+07   
                          Equity(28326 [VNDA])         1.080473e+07   
                          Equity(20359 [EGOV])         4.080842e+06   
                          Equity(27817 [SPWR])         1.458094e+07   
                          Equity(7130 [STAA])          1.115727e+07   

                                                   MarketCap  close_price  \
2019-08-02 00:00:00+00:00 Equity(13698 [MYGN])  3.302539e+09       45.180   
                          Equity(28326 [VNDA])  8.362964e+08       15.800   
                          Equity(20359 [EGOV])  1.515300e+09       22.620   
                          Equity(27817 [SPWR])  2.067699e+09       14.514   
                          Equity(7130 [STAA])   1.550095e+09       34.840   

                                                open_price  ranked_returns  \
2019-08-02 00:00:00+00:00 Equity(13698 [MYGN])       29.23          4138.0   
                          Equity(28326 [VNDA])       15.50          1626.0   
                          Equity(20359 [EGOV])       19.53          6956.0   
                          Equity(27817 [SPWR])       14.01          7194.0   
                          Equity(7130 [STAA])        33.50          5443.0   

                                                sector      volume  
2019-08-02 00:00:00+00:00 Equity(13698 [MYGN])     206   9722361.0  
                          Equity(28326 [VNDA])     206   3006710.0  
                          Equity(20359 [EGOV])     311   1681323.0  
                          Equity(27817 [SPWR])     311  22045883.0  
                          Equity(7130 [STAA])      206   1494931.0

### Describing price over time with a curve
To describe price over time, we'll use integers that increment each day as the independent variable. We'll use price as the dependent variable. Let's practice how to regress the stock price against time antime squared. Ths will allow us to describe the trajectory of proce over time using a polynomial.

$
ClosePrice_i = \beta \times time_i + \gamma \times time_i^2
$

First, we'll use `numpy.arange(days)` where days might be 5 for a week or 252 for a year's worth of data. So we'll have integers represent the days in this window.

To create a 2D numpy array, we can combine them together in a list. By default, the `numpy.arange` arays are row vectors, so we use transpose to make them column vectors (one column for each independent variable).

We instantiate a `LinearRegression` object, then call `.fit(X,y)`, passing in the independent and dependent variables.

We'll use `.coefficient` to access the coefficients estimated from the data. There is one for each independent variable.

In [16]:
# we're choosing a window of 5 days as an example
X = np.array([np.arange(5), np.arange(5)**2])
X = X.T
X

array([[ 0,  0],
       [ 1,  1],
       [ 2,  4],
       [ 3,  9],
       [ 4, 16]])

In [17]:
#we're making up some numbers to represent the stock price
y = np.array(np.random.random(5)*2)
y

array([ 1.17895568,  0.62622185,  1.3010132 ,  0.24211514,  0.80960867])

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
reg = LinearRegression()
reg.fit(X,y);

### Quiz  1
Output the estimates for $\beta$ and $\gamma$

In [20]:
# output the estimates for Beta and gamma
reg.coef_

array([-0.25707016,  0.03619752])

### outputs
`outputs` is a class variable defined in CustomFactor class. We'll set outputs to a list of strings, representing the member variables of the `out` object.

* outputs(iterable[str], optional) - An iterable of strings which represent the names of eac output this factor should compute and return. If this argument is not passed to the CustomFactor constructor, we look for a class-level attribute named outputs.

> So for example, if we create a subclass that inherits from CustomFactor, we can define a class variable `outputs = ['var1', 'var2']`, passing in hte names of the variables as strings

Here's how this variable is used inside the `compute` function:
>out: np.array[self.dtype, ndim=1] Output array of the same shape as `assets`. `compute` should write its desired return values into `out`. if multiple outputs are specified, `compute` should write its desired return values into `out.<output_name>` for each output name in `self.outputs`.

So if we define `outputs = ['var1', 'var2']`, then inside out `compute` function, we'll have `out.var1` and `out.var2` that are numpy arrays. Each of these numpy arrays has one element for each stock that we're processing ( this is done for us by teh code we inherited from CustomFactor)

### numpy.isfinite
Numpy has a way to check for `NaN` (not a number) using `numpy.isnan()`. We can also check if a numer is neither `NaN` nor infinite using `numpy.isfinite()`.

### Quiz 2: Regression against Time
We'll construct a class that inherits from CustomFactor, called `RegressionAgainstTime`. It will perform a regression on one year's worth of daily data at a time. If the stock price is either NaN or infinity (bad data, or an infinitely amazing company!), then we dont want to run it through a regression.
**Hint:** See how we do things for the beta variable, and you can do something similar for the gamma variable.

In [29]:
from quantopian.pipeline.factors import CustomFactor

In [30]:
class RegressionAgainstTime(CustomFactor):

    #TODO: choose a window length that spans one year's worth of trading days
    window_length = 252
    
    #TODO: use USEquityPricing's close price (in a list)
    inputs = [USEquityPricing.close]
    
    #TODO: set outputs to a list of strings, which are names of the outputs
    #We're calculating regression coefficients for two independent variables, 
    # called beta and gamma
    outputs = ['beta', 'gamma']
    
    def compute(self, today, assets, out, dependent):
        
        #TODO: define an independent variable that represents time from the start to end
        # of the window length. E.g. [1,2,3...252]
        #t1 = today
        t1 = np.arange(self.window_length)
        
        #TODO: define a second independent variable that represents time ^2
        #t2 = today ** 2
        t2 = np.arange(self.window_length) **2
        
        # combine t1 and t2 into a 2D numpy array
        X = np.array([t1, t2]).T
    
        #TODO: the number of stocks is equal to the length of the "out" variable,
        # because the "out" variable has one element for each stock
        n_stocks = len(out)
        # loop over each asset

        for i in range(n_stocks):
            # TODO: "dependent" is a 2D numpy array that
            # has one stock series in each column,
            # and days are along the rows.
            # set y equal to all rows for column i of "dependent"
            y = dependent[:,i]
            
            # TODO: run a regression only if all values of y
            # are finite.
            if np.all(np.isfinite(y)):
                # create a LinearRegression object
                regressor = LinearRegression()
                
                # TODO: fit the regressor on X and y
                regressor.fit(X,y)
                
                # store the beta coefficient
                out.beta[i] = regressor.coef_[0]
                
                #TODO: store the gamma coefficient
                out.gamma[i] = regressor.coef_[1]
            else:
                # store beta as not-a-number
                out.beta[i] = np.nan
                
                # TODO: store gammas not-a-number
                out.gamma[i] = np.nan



### Quiz 3: Create a conditional factor
We can create the conditional factor as the product of beta and gamma factors

$
joint_{Factor} = \beta_{Factor}  \times  \gamma_{Factor}
$

if you see the [documentation for the Factor class](https://www.zipline.io/appendix.html?highlight=customfactor#zipline.pipeline.factors.Factor):
> Factors can be combined, both with other Factors and with scalar values, via any of the builtin mathematical operators (+, -, *, etc.) this makes it easy to write complex expressions that combine multi Factors. For example, constructing a Factor that computes the average of two other Factors is simply:

```
f1 = SomeFactor()
f2 = SomeFactor()
average = (f1 + f2) / 2.0
```

In [34]:
# example: we'll call the RegressionAgainstTime constructor,
# pass in the "universe" variable as our mask,
# and get the "beta" variable from that object.
# Then we'll get the rank based on the beta value

universe = AverageDollarVolume(window_length=120).top(500)
###########################################################
def make_pipeline2():
    beta_factor = (
    RegressionAgainstTime(mask = universe).beta.rank()
    )

    # similar to the beta factor, we'll create a gamma factor
    gamma_factor = RegressionAgainstTime(mask=universe).gamma.rank()

    # if we multiply the beta factor and gamma factor,
    # we can then rank that product to creae the conditional factor
    conditional_factor = (beta_factor * gamma_factor).rank()
    
    return Pipeline(
        columns = {
            'time_beta': beta_factor,
            'time_gamma': gamma_factor,
            'conditional_factor': conditional_factor
        },
        screen = universe
    )



In [35]:
QTU_pipeline3 = run_pipeline(make_pipeline2(), start_date, end_date, chunksize=252)

In [36]:
QTU_pipeline3

conditional_factor  time_beta  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                   136.0      372.0   
                          Equity(24 [AAPL])                  262.0      100.0   
                          Equity(62 [ABT])                   283.0      196.0   
                          Equity(64 [GOLD])                  189.0      380.0   
                          Equity(67 [ADSK])                  438.0      169.0   
                          Equity(76 [TAP])                   212.0      411.0   
                          Equity(114 [ADBE])                 440.0      358.0   
                          Equity(122 [ADI])                  425.0      174.0   
                          Equity(128 [ADM])                  410.0      303.0   
                          Equity(161 [AEP])                  213.0      344.0   
                          Equity(168 [AET])                  443.0      296.0   
                          Equity(185 [AFL])                  218.0      407.0   
                          Equity(216 [HES])                  235.0      392.0   
                          Equity(239 [AIG])                  156.0       70.0   
                          Equity(337 [AMAT])                 441.0      241.0   
                          Equity(351 [AMD])                  327.0      140.0   
                          Equity(357 [TWX])                  300.0      104.0   
                          Equity(368 [AMGN])                  55.0      454.0   
                          Equity(448 [APA])                  430.0      376.0   
                          Equity(455 [APC])                  476.0      330.0   
                          Equity(460 [APD])                  332.0      419.0   
                          Equity(630 [ADP])                  429.0      213.0   
                          Equity(679 [AXP])                  330.0      129.0   
                          Equity(693 [AZO])                   36.0      474.0   
                          Equity(698 [BA])                   153.0       65.0   
                          Equity(700 [BAC])                  152.0       72.0   
                          Equity(734 [BAX])                  299.0      353.0   
                          Equity(754 [BBY])                  201.0       85.0   
                          Equity(794 [BDX])                   45.0      475.0   
                          Equity(858 [BHI])                  147.0       64.0   
...                                                            ...        ...   
2019-08-05 00:00:00+00:00 Equity(49655 [TEAM])               476.0      333.0   
                          Equity(50077 [TWLO])               462.0      472.0   
                          Equity(50288 [TTD])                474.0      268.0   
                          Equity(50338 [NTNX])                41.0      407.0   
                          Equity(50350 [COUP])               333.0      116.0   
                          Equity(50524 [UWT])                124.0       70.0   
                          Equity(50683 [SNAP])               376.0      214.0   
                          Equity(50758 [OKTA])               433.0      170.0   
                          Equity(51157 [DD])                   NaN        NaN   
                          Equity(51231 [ROKU])                73.0       32.0   
                          Equity(51314 [MDB])                482.0      464.0   
                          Equity(51325 [SE])                 438.0      243.0   
                          Equity(51653 [VXX])                  NaN        NaN   
                          Equity(51806 [CRON])                57.0      455.0   
                          Equity(51832 [ZS])                 472.0      322.0   
                          Equity(51887 [IQ])                 249.0      224.0   
                          Equity(51895 [SPOT])                29.0       13.0   
                          Equity(52038 [CGC])                 78.0      437.0  